In [ ]:
import numpy as np
from tqdm.auto import tqdm
import networkx as nx
import pickle

In [ ]:
from utils import load_json, compute_rel

In [ ]:
bedroom_sg = load_json("3dp/sgn-data-train/bedroom_data.json")

In [ ]:
#bedroom_info = load_json("metadata/data_rot_train.json")

In [ ]:
#bedroom_sg[3]

In [ ]:
index = np.random.randint(100)

In [ ]:
#bedroom_sg[index]['node_list']

In [ ]:
relation_graph = nx.Graph()

In [ ]:
for index in tqdm(range(len(bedroom_sg))): #len(bedroom_sg)
    G = nx.Graph()
    for obj_key in bedroom_sg[index]['node_list']:
        obj_info = bedroom_sg[index]['node_list'][obj_key]
        obj_vocab = obj_key.split("_")[-1] if not "wall" in obj_key else "wall"
        # print(obj_vocab, obj_info)
        if obj_vocab not in G.nodes:
            G.add_node(obj_vocab)
            G.nodes[obj_vocab]["count"] = 1
            G.nodes[obj_vocab]["surround"] = []
            G.nodes[obj_vocab]["support"] = []
            G.nodes[obj_vocab]["co-occurrence"] = []
        else:
            G.nodes[obj_vocab]["count"] += 1

        if 'support' in obj_info:
            for support_obj_id in obj_info['support']:
                support_obj_vocab = support_obj_id.split('_')[-1]
                G.nodes[obj_vocab]["support"].append(support_obj_vocab)

                if support_obj_vocab not in G.nodes:
                    G.add_node(support_obj_vocab)
                    G.nodes[support_obj_vocab]["count"] = 0
                    G.nodes[support_obj_vocab]["surround"] = []
                    G.nodes[support_obj_vocab]["support"] = []
                    G.nodes[support_obj_vocab]["co-occurrence"] = []

                G.add_edge(obj_vocab, support_obj_vocab)

        if 'surround' in obj_info and len(obj_info['surround']) > 0:
            # print("SURROUND", obj_key, obj_info)
            for surround_obj_id in obj_info['surround']:
                surround_obj_vocab = list(surround_obj_id.keys())[0].split('_')[-1]
                G.nodes[obj_vocab]["surround"].append(surround_obj_vocab)

                if surround_obj_vocab not in G.nodes:
                    G.add_node(surround_obj_vocab)
                    G.nodes[surround_obj_vocab]["count"] = 0
                    G.nodes[surround_obj_vocab]["surround"] = []
                    G.nodes[surround_obj_vocab]["support"] = []
                    G.nodes[surround_obj_vocab]["co-occurrence"] = []

                G.add_edge(obj_vocab, surround_obj_vocab)

        if 'co-occurrence' in obj_info and len(obj_info['co-occurrence']) > 0:
            for cooccurrence_obj_id in obj_info['co-occurrence']:
                cooccurrence_obj_vocab = cooccurrence_obj_id.split('_')[-1] if not "wall" in cooccurrence_obj_id else "wall"
                if cooccurrence_obj_vocab not in G.nodes[obj_vocab]["co-occurrence"]:
                    G.nodes[obj_vocab]["co-occurrence"].append(cooccurrence_obj_vocab)

                if cooccurrence_obj_vocab not in G.nodes:
                    G.add_node(cooccurrence_obj_vocab)
                    G.nodes[cooccurrence_obj_vocab]["count"] = 0
                    G.nodes[cooccurrence_obj_vocab]["co-occurrence"] = []
                    G.nodes[cooccurrence_obj_vocab]["surround"] = []
                    G.nodes[cooccurrence_obj_vocab]["support"] = []

                G.add_edge(obj_vocab, cooccurrence_obj_vocab)

    for node in G.nodes:
        for obj1 in G.nodes[node]["co-occurrence"]:
            for obj2 in G.nodes[node]["co-occurrence"]:
                # add node and edge
                if obj1 not in relation_graph.nodes:
                    relation_graph.add_node(obj1)

                if obj2 not in relation_graph.nodes:
                    relation_graph.add_node(obj2)

                if (obj1, obj2) not in relation_graph.edges:
                    relation_graph.add_edge(obj1, obj2)
                    relation_graph.edges[(obj1,obj2)]['count'] = 1
                else:
                    relation_graph.edges[(obj1,obj2)]['count'] += 1

        for obj1 in G.nodes[node]["support"]:
            for obj2 in G.nodes[node]["support"]:
                # add node and edge
                if obj1 not in relation_graph.nodes:
                    relation_graph.add_node(obj1)

                if obj2 not in relation_graph.nodes:
                    relation_graph.add_node(obj2)

                if (obj1, obj2) not in relation_graph.edges:
                    relation_graph.add_edge(obj1, obj2)
                    relation_graph.edges[(obj1,obj2)]['count'] = 1
                else:
                    relation_graph.edges[(obj1,obj2)]['count'] += 1
    

In [ ]:
relation_graph.edges('bed')

In [ ]:
relation_graph.nodes()

In [ ]:
for edge in relation_graph.edges('bed'):
    print(edge, ":", relation_graph.edges[edge]['count'])

In [ ]:
relation_graph.out_edges('bed')

In [ ]:
pickle.dump(relation_graph, open("new/relation_graph_v1.p", "wb"))

In [ ]:
"shower_curtain" in relation_graph.nodes